# Final Project

For the final project, your task is to predict the type of galaxies based on a number of measurements and calculations about the images of the galaxies.  This is similar to the [Galaxy Zoo](http://zoo1.galaxyzoo.org/) project we discussed in class, but rather than starting from the images themselves, we are starting with some properties of the galaxies measured from the images.

The data are fully described in [Fischer et al (2019)](https://arxiv.org/abs/1811.02580), a recent paper by several U. Penn researchers, including Helena Dominguez-Sanchez, whom you met in class on April 17.
Don't worry.  You don't need to read that paper to do this project.  Helena explained the important aspects of the data in class.  (Her slides are available on [canvas](https://canvas.upenn.edu/files/76244434/).)  But if you feel inspired to read more about it, please do take a look at the paper.

The Fischer et al paper describes quite a lot of measurements based on different kinds of fits to the images.  We have extracted a subset of the measurements into a smaller catalog, which should be easier for you to work with.  Specifically, we selected columns related to the bulge + disk decomposition of the galaxy (called SE for Sersic + Exponential in the paper).

The columns in this catalog are:

* id = A numeric ID identifying the galaxy
* gal_type = Which type of galaxy this is.  We discuss this more below.
* ra = Right Ascension in degrees.  Kind of like longitude on the sky.
* dec = Declination in degrees.  Like latitude on the sky.
* redshift = A measure of how far away the galaxy is.
* log_luminosity = Base 10 logarithm of the galaxy's brightness (aka luminosity) measured as a multiple of the sun's luminosity.
* color = Lr / Lg, the ratio of the luminosity at red wavelengths to the luminosity at green wavelengths.
* radius = The half-light radius of the galaxy (called Re in Helena's slides) according to the bulge + disk fit in kiloparsecs (kpc; see below for definition of kpc).
* b_over_a = The ratio of the semi-minor axis of the galaxy (b) to the semi-major axis (a).  b_over_a = 1 is a circle. b_over_a close to 0 is a very elongated ellipse.
* pos_angle = The position angle of the semi-minor axis of the galaxy on the sky in degrees.
* bulge_fract = The fraction of the total light that was found to be in the bulge component (called B/T in Helena's slides).
* sersic_n = The Sersic index of the bulge component of the fit.
* r_bulge = The half-light radius of the bulge component of the fit in kiloparsecs (kpc).
* r_disk = The half-light radius of the disk component of the fit in kiloparsecs (kpc).

Radii are given in kpc above, a common distance unit used by astronomers. For reference, 1 kpc = 1000 pc, and 1 pc = 3.26 light years.

--- 

The gal_type column is your target (y).  The others you may use as your predictors (x_i).

For the galaxy type, we distilled the morphological classification described in Fischer et al into 4 broad categories.

* gal_type = 1 are elliptical galaxies.  In the paper, these are called E.  They have TTYPE <= 0 and P_S0 <= 0.5.
* gal_type = 2 are lenticular galaxies.  In the paper, these are called S0.  They have TTYPE <= 0 and P_S0 > 0.5.
* gal_type = 3 are tight spiral galaxies.  In the paper, these are described as 0 < TType < 3.  We will call them S1.
* gal_type = 4 are loose spiral galaxies.  In the paper, these are described as TType > 3.  We will call them S2.

Note: in addition to the catalog described above, we are also providing the complete catalog described in Fischer et al for g and r bands.  You are not required to use it, but you may do so if you want.

## Part 1

For the first stage of the project, we'll start with the easier task of just distinguishing the two most extreme galaxy types: 1 vs 4, or E vs S2.

You may use any of the various characterization methods we have used throughout the course to do the prediction.  (Or even ones we haven't covered if you feel so inclined.)  You should write two functions:

* train_galaxy_predictions takes as input a set of training data and returns some results, which can be used to make predictions.
* validate_galaxy_predictions takes the results from the first function and some validation data to predict the galaxy types.  It should return n array of predicted galaxy types.

You can decide how you want to split the provided data into training and validation samples.  (When you submit your project, we will run your code using the full given data as the training set.  We have reserved a validation sample of approximately equal size to use for the validation step.)

In [22]:
#import necessary modules and functions
import numpy as np
import pandas as pd
import keras
from sklearn.preprocessing import StandardScaler as scale
from sklearn.metrics import confusion_matrix
from keras.layers import Input, Dense, Dropout
from keras.models import Sequential

def train_galaxy_predictions_1(gal_type, data, data_g, data_r):
    """Do some kind of training to learn how to predict the galaxy type from the given training data.
    """
    
    #seed predictor data and response into dataframes for easier manipulation
    predictor = pd.DataFrame(gal_type)    
    df_data = pd.DataFrame(data)
    
    #delete unwanted columns, i.e. false predictors
    df_data = df_data.drop(['id', 'gal_type'], axis= 1)

    #convert back from dataframes to arrays since that is what functions as input in keras
    train = df_data.values
    train_response = predictor.values
    
    #since this is a binary classification problem, assign a value of 0 to one of the response possibilities (galaxy type 1)
    #and 1 for the other (galaxy type 4). this is called one-hot encoding.
    train_response[train_response == 1] = 0
    train_response[train_response == 4] = 1
   
    #scale the predictor data accordingly.
    #scale function applies a transformation that results in a standard normal distribution ~ Z(0,1)
    sc = scale()
    scale_train = sc.fit_transform(train)
    
    #build the tree with dropout layers to prevent overfitting
    classifier = Sequential()
    classifier.add(Dense(input_shape = (df_data.values.shape[1],), units= 10, kernel_initializer= 'uniform', activation = 'relu'))
    classifier.add(Dropout(0.2, noise_shape=None, seed=None))
    classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'linear'))
    classifier.add(Dropout(0.2, noise_shape=None, seed=None))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    
    #compile tree with chosen optimizer and loss function
    #binary cross-entropy is chosen as this is a binary classification problem
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #train neural net
    classifier.fit(scale_train, train_response, batch_size = 3, epochs = 15)
    
    #return neural net as an object for use
    return(classifier)

def validate_galaxy_predictions_1(result, data, data_g, data_r):
    """Use the output from the above function to predict the galaxy type for some other validation data.
    """
    #use neural net that resulted from running train_galaxy_predictions method
    classifier = result
    
    #seed predictor data into dataframe for easier manipulation
    df_data = pd.DataFrame(data)
    
    #delete unwanted columns, i.e. false predictors
    df_data = df_data.drop(['id', 'gal_type'], axis= 1)
    
    #convert back from dataframes to arrays since that is what functions as input in keras
    test = df_data.values
    
    #scale the predictor data accordingly.
    #scale function applies a transformation that results in a standard normal distribution ~ Z(0,1)
    sc = scale()
    scale_test = sc.fit_transform(test)
    
    #predict which class the data is in
    y_pred = classifier.predict(scale_test)
    
    #assign class based on convention introduced in creation of tree.
    y_pred[y_pred >= .5] = 4
    y_pred[y_pred < .5] = 1
    
    #reformat
    y_pred = y_pred.reshape((y_pred.shape[0],))
    
    #return predictor values
    return(y_pred)

In [23]:
import numpy as np

gal_file_name = 'training_galaxies.dat'
full_g_file_name = 'training_galaxies_full_g.dat'
full_r_file_name = 'training_galaxies_full_r.dat'

gal_data = np.genfromtxt(gal_file_name, names=True, dtype=None)
full_g_data = np.genfromtxt(full_g_file_name, names=True, dtype=None)
full_r_data = np.genfromtxt(full_r_file_name, names=True, dtype=None)

# For now, limit the data to those galaxies with gal_type == 1 or 4
gal_type = gal_data['gal_type'].copy()
type_1_4 = (gal_type == 1) | (gal_type == 4)
gal_data = gal_data[type_1_4]
full_g_data = full_g_data[type_1_4]
full_r_data = full_r_data[type_1_4]
gal_type = gal_type[type_1_4]

# Zero out the gal_type column in the input data, so it can't be used to cheat.  :)
gal_data['gal_type'][:] = 0

# Split up the input into a training and validation set as you want.
# When we run this, we will use the full data arrays as input, and some different (not provided!) data as validation.
### EDIT THIS CODE...

#generalize so you are aable to choose various fractions of available data to run through neural net
n=1000
training_data = gal_data[:n]
training_data_g = full_g_data[:n]
training_data_r = full_r_data[:n]
training_gal_type = gal_type[:n]
validation_data = gal_data[n:]
validation_data_g = full_g_data[n:]
validation_data_r = full_r_data[n:]
validation_gal_type = gal_type[n:]

# Your function can ignore training_data_g and training_data_r if you don't want to use them.
# But the function signature needs to take 4 values, since we will pass them to your function when we run this.
result = train_galaxy_predictions_1(training_gal_type, training_data, training_data_g, training_data_r)
predicted_gal_type = validate_galaxy_predictions_1(result, validation_data, validation_data_g, validation_data_r)

#run predict function on the training set to see how well it does (see confusion matrix for training data set below)
predict_train = validate_galaxy_predictions_1(result, training_data, training_data_g, training_data_r)

print('Num correct = ', np.sum(predicted_gal_type == validation_gal_type))
print('Num total = ',len(validation_gal_type))
print('Fraction correct = ',np.sum(predicted_gal_type == validation_gal_type) / len(validation_gal_type))
# Note: It may be helpful to construct a confusion matrix to better quantify how well you are doing here, rather
# than just looking at the overal fraction correct.

#create confusion matrix for test
confusion_test = confusion_matrix(predicted_gal_type, validation_gal_type)

#create confusion matrix for train data set
training_gal_type[training_gal_type == 1] = 4
training_gal_type[training_gal_type == 0] = 1
confusion_train = confusion_matrix(predict_train, training_gal_type)

#print out confusion matrices
print("Test is: ")
print(confusion_test)
print("Train is: ")
print(confusion_train)

Epoch 1/15
1000/1000 [==============================] - 1s 1ms/step - loss: 0.3868 - acc: 0.8630
Epoch 2/15
1000/1000 [==============================] - 0s 354us/step - loss: 0.1305 - acc: 0.9550
Epoch 3/15
1000/1000 [==============================] - 0s 374us/step - loss: 0.1111 - acc: 0.9610
Epoch 4/15
1000/1000 [==============================] - 0s 367us/step - loss: 0.0958 - acc: 0.9660
Epoch 5/15
1000/1000 [==============================] - 0s 481us/step - loss: 0.0957 - acc: 0.9700
Epoch 6/15
1000/1000 [==============================] - 0s 477us/step - loss: 0.0934 - acc: 0.9700
Epoch 7/15
1000/1000 [==============================] - 0s 476us/step - loss: 0.0871 - acc: 0.9710
Epoch 8/15
1000/1000 [==============================] - 0s 371us/step - loss: 0.0919 - acc: 0.9680
Epoch 9/15
1000/1000 [==============================] - 0s 484us/step - loss: 0.0918 - acc: 0.9720
Epoch 10/15
1000/1000 [==============================] - 0s 374us/step - loss: 0.0909 - acc: 0.9720
Epoch 11/15

## Part 2

The second stage of the project is the same, except we will try to classify the spirals into their two sub-categories, S1 vs S2 (i.e. types 3 vs 4).

In [24]:
#import necessary modules and functions
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler as scale
from sklearn.metrics import confusion_matrix
from keras.layers import Input, Dense, Dropout
from keras.models import Sequential

def train_galaxy_predictions_2(gal_type, data, data_g, data_r):
    """Do some kind of training to learn how to predict the galaxy type from the given training data.
    """
    
    #seed predictor data and response into dataframes for easier manipulation
    predictor = pd.DataFrame(gal_type)    
    df_data = pd.DataFrame(data)
    
    #delete unwanted columns, i.e. false predictors
    df_data = df_data.drop(['id', 'gal_type'], axis= 1)
    
    #convert back from dataframes to arrays since that is what functions as input in keras
    train = df_data.values
    train_response = predictor.values
    
    #since this is a binary classification problem, assign a value of 0 to one of the response possibilities (galaxy type 3)
    #and 1 for the other (galaxy type 4). this is called one-hot encoding.
    train_response[train_response == 3] = 0
    train_response[train_response == 4] = 1
    
    #scale the predictor data accordingly.
    #scale function applies a transformation that results in a standard normal distribution ~ Z(0,1)
    sc = scale()
    scale_train = sc.fit_transform(train)
    
    #build the tree with dropout layers to prevent overfitting
    classifier = Sequential()
    classifier.add(Dense(input_shape = (df_data.values.shape[1],), units= 16, kernel_initializer= 'uniform', activation = 'relu'))
    classifier.add(Dropout(0.5, noise_shape=None, seed=None))
    classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'linear'))
    classifier.add(Dropout(0.5, noise_shape=None, seed=None))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    
    #compile tree with chosen optimizer and loss function
    #binary cross-entropy is chosen as this is a binary classification problem
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #train neural net
    classifier.fit(scale_train, train_response, batch_size = 3, epochs = 15)
    
    #return neural net as an object for use
    return(classifier)

def validate_galaxy_predictions_2(result, data, data_g, data_r):
    """Use the output from the above function to predict the galaxy type for some other validation data.
    """
    
    #use neural net that resulted from running train_galaxy_predictions_2 method
    classifier = result
    
    #seed predictor data into dataframe for easier manipulation
    df_data = pd.DataFrame(data)
    
    #delete unwanted columns, i.e. false predictors
    df_data = df_data.drop(['id', 'gal_type'], axis= 1)
    
    #convert back from dataframes to arrays since that is what functions as input in keras
    test = df_data.values
    
    #scale the predictor data accordingly.
    #scale function applies a transformation that results in a standard normal distribution ~ Z(0,1)
    sc = scale()
    scale_test = sc.fit_transform(test)
    
    #predict which class the data is in
    y_pred = classifier.predict(scale_test)
    
    #assign class based on convention introduced in creation of tree.
    y_pred[y_pred >= .5] = 4
    y_pred[y_pred < .5] = 3
     
    #reformat
    y_pred = y_pred.reshape((y_pred.shape[0],))
    
    #return predictor values
    return(y_pred)

In [25]:
import numpy as np

gal_file_name = 'training_galaxies.dat'
full_g_file_name = 'training_galaxies_full_g.dat'
full_r_file_name = 'training_galaxies_full_r.dat'

gal_data = np.genfromtxt(gal_file_name, names=True, dtype=None)
full_g_data = np.genfromtxt(full_g_file_name, names=True, dtype=None)
full_r_data = np.genfromtxt(full_r_file_name, names=True, dtype=None)

# This time, limit the data to those galaxies with gal_type == 3 or 4
gal_type = gal_data['gal_type'].copy()
type_3_4 = (gal_type == 3) | (gal_type == 4)
gal_data = gal_data[type_3_4]
full_g_data = full_g_data[type_3_4]
full_r_data = full_r_data[type_3_4]
gal_type = gal_type[type_3_4]

# Zero out the gal_type column in the input data, so it can't be used to cheat.  :)
gal_data['gal_type'][:] = 0

# Split up the input into a training and validation set as you want.
# When we run this, we will use the full data arrays as input, and some different (not provided!) data as validation.
### EDIT THIS CODE...

#generalize so you are able to choose various fractions of available data to run through neural net
n=1000
training_data = gal_data[:n]
training_data_g = full_g_data[:n]
training_data_r = full_r_data[:n]
training_gal_type = gal_type[:n]
validation_data = gal_data[n:]
validation_data_g = full_g_data[n:]
validation_data_r = full_r_data[n:]
validation_gal_type = gal_type[n:]

# Your function can ignore training_data_g and training_data_r if you don't want to use them.
# But the function signature needs to take 4 values, since we will pass them to your function when we run this.
result = train_galaxy_predictions_2(training_gal_type, training_data, training_data_g, training_data_r)
predicted_gal_type = validate_galaxy_predictions_2(result, validation_data, validation_data_g, validation_data_r)

#run predict function on the training set to see how well it does (see confusion matrix for training data set below)
predict_train2 = validate_galaxy_predictions_2(result, training_data, training_data_g, training_data_r)

print('Num correct = ',np.sum(predicted_gal_type == validation_gal_type))
print('Num total = ',len(validation_gal_type))
print('Fraction correct = ',np.sum(predicted_gal_type == validation_gal_type) / len(validation_gal_type))
# Note: It may be helpful to construct a confusion matrix to better quantify how well you are doing here, rather
# than just looking at the overal fraction correct.

#create confusion matrix for test data set
confusion_test = confusion_matrix(predicted_gal_type, validation_gal_type)

#create confusion matrix for train data set
training_gal_type[training_gal_type == 1] = 4
training_gal_type[training_gal_type == 0] = 3
confusion_train = confusion_matrix(predict_train2, training_gal_type)

#print out confusion matrices
print("Test is: ")
print(confusion_test)
print("Train is: ")
print(confusion_train)

Epoch 1/15
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6403 - acc: 0.6240
Epoch 2/15
1000/1000 [==============================] - 0s 446us/step - loss: 0.5040 - acc: 0.7410
Epoch 3/15
1000/1000 [==============================] - 0s 459us/step - loss: 0.4971 - acc: 0.7680
Epoch 4/15
1000/1000 [==============================] - 0s 365us/step - loss: 0.4816 - acc: 0.7870
Epoch 5/15
1000/1000 [==============================] - 0s 349us/step - loss: 0.4539 - acc: 0.8150
Epoch 6/15
1000/1000 [==============================] - 0s 348us/step - loss: 0.4745 - acc: 0.8030
Epoch 7/15
1000/1000 [==============================] - 0s 347us/step - loss: 0.4677 - acc: 0.8050
Epoch 8/15
1000/1000 [==============================] - 0s 350us/step - loss: 0.4844 - acc: 0.8050
Epoch 9/15
1000/1000 [==============================] - 0s 356us/step - loss: 0.4718 - acc: 0.8100
Epoch 10/15
1000/1000 [==============================] - 0s 347us/step - loss: 0.4777 - acc: 0.7990
Epoch 11/15

## Part 3

Finally, in the third stage, we will try to classify all 4 types.

You may find it simpler to do this in two steps.  First separate E and S0 galaxies (aka, types 1 and 2) from spiral galaxies (S1 and S2, aka types 3 and 4).  Then within these two groups classify E vs S0 and S1 vs S2.  This would allow you to use classifiers that are best done on binary classification.  

Or you may want to try to use a classifier that can handle more than 2 types for the target and do the whole process at once.  Or you may find that some other combination of methods is appropriate.

It should be noted that we don't expect you to be able to do a great job separating the E and S0 galaxies from each other.  So don't feel too bad if you aren't getting great results on that part of the classification.  However, we think you should be able to do a decent job separating the other types from each other.

In [26]:
#import necessary modules and functions
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler as scale
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input, Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical

def train_galaxy_predictions_3(gal_type, data, data_g, data_r):
    """Do some kind of training to learn how to predict the galaxy type from the given training data.
    """
    
    #seed predictor data and response into dataframes for easier manipulation
    predictor = pd.DataFrame(gal_type)    
    df_data = pd.DataFrame(data)
    
    #delete unwanted columns, i.e. false predictors
    df_data = df_data.drop(['id', 'gal_type'], axis= 1)
    
    #convert back from dataframes to arrays since that is what functions as input in keras
    train = df_data.values
    response = predictor.values
    
    #since this is a categorical classification problem, instead of assigning binary values, use built-in encoder to create
    #the necessary one-hot encoded vectors for the 4 types of galaxies. in this case, the output is permutations of 1 and 0s
    #for the number of possible response variables (classes), like [1,0,0,0], [0,1,0,0], etc.
    encoder = LabelEncoder()
    encoder.fit(response)
    encoded_Y = encoder.transform(response)    
    train_response = to_categorical(encoded_Y)
    
    #scale the predictor data accordingly.
    #scale function applies a transformation that results in a standard normal distribution ~ Z(0,1)
    sc = scale()
    scale_train = sc.fit_transform(train)
    
    #build the tree with dropout layers to prevent overfitting
    classifier = Sequential()
    classifier.add(Dense(input_shape = (df_data.values.shape[1],), units= 16, kernel_initializer= 'uniform', activation = 'relu'))
    classifier.add(Dropout(0.3, noise_shape=None, seed=None))
    classifier.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(0.3, noise_shape=None, seed=None))
    classifier.add(Dense(units = 4, activation = 'softmax'))
    
    #compile tree with chosen optimizer and loss function
    #categorical cross-entropy is chosen as this is a categorical classification problem
    classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    #train neural net
    classifier.fit(scale_train, train_response, batch_size = 6, epochs = 30)
    
    #return neural net as an object for use
    return(classifier)

def validate_galaxy_predictions_3(result, data, data_g, data_r):
    """Use the output from the above function to predict the galaxy type for some other validation data.
    """
    
    #use neural net that resulted from running train_galaxy_predictions_3 method
    classifier = result
    
    #seed predictor data into dataframe for easier manipulation
    df_data = pd.DataFrame(data)
    
    #delete unwanted columns, i.e. false predictors
    df_data = df_data.drop(['id', 'gal_type'], axis= 1)
    
    #convert back from dataframes to arrays since that is what functions as input in keras
    test = df_data.values
    
    #scale the predictor data accordingly.
    #scale function applies a transformation that results in a standard normal distribution ~ Z(0,1)
    sc = scale()
    scale_test = sc.fit_transform(test)
    
    #predict which class the data is in
    y_pred = classifier.predict_classes(scale_test)
    
    #assign proper values based on the result of the one-hot encoding
    #the numbers 0,1,2,3 refer to the placement of the 1 in the vectors [1,0,0,0], [0,1,0,0], etc.
    y_pred[y_pred == 3] = 4
    y_pred[y_pred == 2] = 3
    y_pred[y_pred == 1] = 2
    y_pred[y_pred == 0] = 1
    
    #reformat
    y_pred = y_pred.reshape((y_pred.shape[0],))
    
    #return predictor values
    return(y_pred)

import numpy as np

gal_file_name = 'training_galaxies.dat'
full_g_file_name = 'training_galaxies_full_g.dat'
full_r_file_name = 'training_galaxies_full_r.dat'

gal_data = np.genfromtxt(gal_file_name, names=True, dtype=None)
full_g_data = np.genfromtxt(full_g_file_name, names=True, dtype=None)
full_r_data = np.genfromtxt(full_r_file_name, names=True, dtype=None)

# Zero out the gal_type column in the input data, so it can't be used to cheat.  :)
gal_type = gal_data['gal_type'].copy()
gal_data['gal_type'][:] = 0

# Split up the input into a training and validation set as you want.
# When we run this, we will use the full data arrays as input, and some different (not provided!) data as validation.
### EDIT THIS CODE...

#generalize so you are able to choose various fractions of available data to run through neural net
n=1000
training_data = gal_data[:n]
training_data_g = full_g_data[:n]
training_data_r = full_r_data[:n]
training_gal_type = gal_type[:n]
validation_data = gal_data[n:]
validation_data_g = full_g_data[n:]
validation_data_r = full_r_data[n:]
validation_gal_type = gal_type[n:]

# Your function can ignore training_data_g and training_data_r if you don't want to use them.
# But the function signature needs to take 4 values, since we will pass them to your function when we run this.
result = train_galaxy_predictions_3(training_gal_type, training_data, training_data_g, training_data_r)
predicted_gal_type = validate_galaxy_predictions_3(result, validation_data, validation_data_g, validation_data_r)

#run predict function on the training set to see how well it does (see confusion matrix for training data set below)
predict_train3 = validate_galaxy_predictions_3(result, training_data, training_data_g, training_data_r)

print('Num correct = ',np.sum(predicted_gal_type == validation_gal_type))
print('Num total = ',len(validation_gal_type))
print('Fraction correct = ',np.sum(predicted_gal_type == validation_gal_type) / len(validation_gal_type))
# Note: It may be helpful to construct a (4x4) confusion matrix to better quantify how well you are doing here, rather
# than just looking at the overal fraction correct.

#create confusion matrix for test data set
confusion_test = confusion_matrix(predicted_gal_type, validation_gal_type)

#create confusion matrix for train data set
confusion_train = confusion_matrix(predict_train3, training_gal_type)

#print confusion matrices
print("Test is: ")
print(confusion_test)
print("Train is: ")
print(confusion_train)

C:\Users\diana\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\diana\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/30
1000/1000 [==============================] - 1s 1ms/step - loss: 1.2265 - acc: 0.4410
Epoch 2/30
1000/1000 [==============================] - 0s 264us/step - loss: 0.9459 - acc: 0.6400
Epoch 3/30
1000/1000 [==============================] - 0s 312us/step - loss: 0.8473 - acc: 0.6690
Epoch 4/30
1000/1000 [==============================] - 0s 295us/step - loss: 0.8144 - acc: 0.6820
Epoch 5/30
1000/1000 [==============================] - 0s 310us/step - loss: 0.7764 - acc: 0.7130
Epoch 6/30
1000/1000 [==============================] - 0s 191us/step - loss: 0.7736 - acc: 0.7040
Epoch 7/30
1000/1000 [==============================] - 0s 268us/step - loss: 0.7498 - acc: 0.7010
Epoch 8/30
1000/1000 [==============================] - 0s 195us/step - loss: 0.7709 - acc: 0.7030
Epoch 9/30
1000/1000 [==============================] - 0s 269us/step - loss: 0.7379 - acc: 0.7180
Epoch 10/30
1000/1000 [==============================] - 0s 193us/step - loss: 0.7362 - acc: 0.7200
Epoch 11/30